<a href="https://colab.research.google.com/github/dislu/Machine_Learning/blob/main/conv_lstm_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Next-frame prediction with Conv-LSTM

**Author:** [Arvind Singh](https://github.com/jeammimi)<br>
**Date created:** 2021/04/07<br>
**Last modified:** 2021/04/08<br>
**Description:** Predict the next frame in a sequence using a Conv-LSTM model.

## Introduction

This script demonstrates the use of a convolutional LSTM model.
The model is used to predict the next frame of a movie generated from sst daily mean image dataset.


## Setup


In [ ]:
from google.colab import drive
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling3D
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import model_from_json 
import os

drive.mount('/content/drive')
n_samples =12000 # total Number of movie samples 
n_frames = 20    # Number of frames in a movie


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Build a model

We create a model which take as input movies of shape
`(n_frames, width, height, channels)` and returns a movie
of identical shape.


In [ ]:
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                  input_shape=(None, 60, 48, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
#seq.add(MaxPooling3D(pool_size = (1,2,2),padding = 'same'))

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
#seq.add(MaxPooling3D(pool_size = (1,2,2),padding = 'same'))

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
#seq.add(MaxPooling3D(pool_size = (1,2,2), padding = 'same'))

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
#seq.add(MaxPooling3D(pool_size = (1,2,2), padding = 'same'))

seq.add(Conv3D(filters=3, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')



## Generate artificial data

Generate movies with 20 days daily mean images of SST inside.
The squares are of shape 1x1 or 2x2 pixels,
and move linearly over time.
For convenience, we first create movies with bigger width and height (80x80)
and at the end we select a 40x40 window.


In [ ]:

def generate_movies(n_samples, n_frames):
    folder = '/content/drive/My Drive/Colab_Notebooks/sst.day.mean.1981-2019.Figures/'
    images = sorted(os.listdir(folder))
    input_movie=[]
    label_movie=[]
    movie_array_input = []
    movie_array_label = []
    for sample in range(n_samples):
        for frame in range(sample,sample+n_frames):
            # reading images for input data and labels
            im_i = Image.open(folder + images[sample])
            im_l = Image.open(folder + images[sample+1])
            
            #im_i = im_i.getdata()
            #im_l = im_l.getdata()
            # converting images into nparray and append
            movie_array_input.append(np.array(im_i)) #.transpose(1, 0, 2))
            movie_array_label.append(np.array(im_l))
            # converting list of image array to array 
        movie_array_input = np.array(movie_array_input)
        movie_array_label = np.array(movie_array_label)

        input_movie.append(movie_array_input)
        label_movie.append(movie_array_label) 
        movie_array_input = []
        movie_array_label = []
    input_movie = np.array(input_movie)
    label_movie = np.array(label_movie)
    print(input_movie.shape)
    print(label_movie.shape)
    #(75, 50, 100, 3)
    return input_movie, label_movie



## Train the model


In [ ]:
noisy_movies, shifted_movies = generate_movies(n_samples, n_frames)
seq.fit(
    noisy_movies[:int(0.8*n_samples)],
    shifted_movies[:int(0.8*n_samples)],
    batch_size=128,
    epochs=300,
    validation_split=0.33
    )

# save model
# serialize model to JSON
#model_json = seq.to_json()
#with open("seq.json", "w") as json_file:
#     json_file.write(model_json)
# serialize weights to HDF5
#seq.save_weights("model.h5")
#print("Saved model to disk")


(12000, 20, 60, 48, 3)
(12000, 20, 60, 48, 3)
Epoch 1/300


ResourceExhaustedError: ignored

## Test the model on one movie

Feed it with the first 7 positions and then
predict the new positions.


In [ ]:
which = int(0.8*n_samples+10)
track = noisy_movies[which][:int(n_frames/2), ::, ::, ::]

for j in range(n_frames+1):
    new_pos = seq.predict(track[np.newaxis, ::, ::, ::, ::])
    new = new_pos[::, -1, ::, ::, ::]
    track = np.concatenate((track, new), axis=0)


# And then compare the predictions
# to the ground truth
track2 = noisy_movies[which][::, ::, ::, ::]
for i in range(n_frames):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(121)

    if i >= int(n_frames/2):
        ax.text(1, 3, 'Predictions !', fontsize=20, color='w')
    else:
        ax.text(1, 3, 'Initial trajectory', fontsize=20)

    toplot = track[i, ::, ::, 0]

    plt.imshow(toplot)
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', fontsize=20)

    toplot = track2[i, ::, ::, 0]
    if i >= 2:
        toplot = shifted_movies[which][i - 1, ::, ::, 0]

    plt.imshow(toplot)
    plt.savefig('%i_sst_field.png' % (i + 1))
